In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [ ]:


base_dir = Path("/Users/cristianrossato/Desktop/social/code_3_11")
ospiti_file = base_dir / "progetto/data/gemini_data/ospiti_cleared(6).csv"
episodi_file = base_dir / "progetto/data/scripts/yt/yt_data/joe_rogan_episodes.csv"

output_dir = base_dir / "progetto/data"
output_file = output_dir / "associazione_guest_visual.csv"

df_ospiti = pd.read_csv(ospiti_file)

df_episodi = pd.read_csv(episodi_file)


In [ ]:
def normalizza_nome(nome):
    if pd.isna(nome):
        return ""
    nome = str(nome).lower().strip()
    nome = re.sub(r'[^a-zA-Z0-9\s]', '', nome)
    nome = re.sub(r'\s+', ' ', nome)
    return nome

df_ospiti['nome_normalizzato'] = df_ospiti['nome'].apply(normalizza_nome)
df_episodi['guest_normalizzato'] = df_episodi['guest'].apply(normalizza_nome)


In [ ]:
def nome_contenuto_in_guest(nome_ospite, nome_guest):
    if not nome_ospite or not nome_guest:
        return False
    
    return nome_ospite in nome_guest

risultati = []


for idx, ospite in df_ospiti.iterrows():
    nome_ospite = ospite['nome_normalizzato']
    nome_originale = ospite['nome']
    
    episodi_ospite = df_episodi[df_episodi['guest_normalizzato'].apply(
        lambda x: nome_contenuto_in_guest(nome_ospite, x)
    )]
    
    if len(episodi_ospite) > 0:
        for _, episodio in episodi_ospite.iterrows():
            risultati.append({
                'nome': nome_originale,
                'video_id': episodio['video_id'],
                'title': episodio['title'],
                'published_at': episodio['published_at'],
                'views': episodio['views'],
                'likes': episodio['likes'],
                'comments': episodio['comments']
            })
        

In [ ]:
if risultati:
    df_risultati = pd.DataFrame(risultati)
    
    df_risultati = df_risultati.sort_values('views', ascending=False)
    
    output_dir.mkdir(exist_ok=True)
    
    df_risultati_finale = df_risultati[['nome', 'video_id','published_at', 'views', 'likes', 'comments']]
    df_risultati_finale.to_csv(output_file, index=False)


In [ ]:
if risultati:
    
    ospiti_stats = df_risultati.groupby('nome').agg({
        'views': ['count', 'mean', 'max'],
        'likes': 'mean',
        'comments': 'mean'
    }).round(0)
    
    ospiti_stats.columns = ['episodi', 'views_media', 'views_max', 'likes_media', 'comments_media']
    ospiti_stats = ospiti_stats.sort_values('views_media', ascending=False)